## Requesting Instrument Asset Information

In this brief demo you will learn:

* how to request asset information for an instrument, such as deployment start and end dates, lat, lon and depth

Before we get started, login in at https://ooinet.oceanobservatories.org/ and obtain your <b>API username and API token</b> under your profile (top right corner).

In [1]:
username = ''
token = ''

Optionally, you can handle authentication outside the notebook by setting up a .netrc file in your home directory and loading it with your bash profile. Open your terminal
```
$ touch .netrc
$ chmod 700 .netrc
$ vim .netrc

```
Add the following your your .netrc file:

```
machine ooinet.oceanobservatories.org
login OOIAPI-TEMPD1SPK4K0X
password TEMPCXL48ET2XT
```

Use your username and token. Save the file and uncomment the following cell.

In [2]:
import netrc
netrc = netrc.netrc()
remoteHostName = "ooinet.oceanobservatories.org"
info = netrc.authenticators(remoteHostName)
username = info[0]
token = info[2]

### Pulling asset information into your notebook.

The ingredients being used to build the asset_info_url can be found here. For this example, we will pull in the asset information for the 3D Thermistor Array http://ooi.visualocean.net/instruments/view/RS03ASHS-MJ03B-07-TMPSFA301

In [3]:
import requests
import datetime
import pandas as pd

In [12]:
subsite = 'RS03AXPS'
node = 'PC03A'
sensor = '4A-CTDPFA303'

Now let's build the asset_info_url and request the information

In [13]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12587/events/deployment/inv/'

asset_info_url ='/'.join((base_url, subsite, node, sensor, '-1'))

r = requests.get(asset_info_url,auth=(username, token))
asset_info = r.json()

In [14]:
asset_info[0]['location']

{u'depth': 190.0,
 u'latitude': 45.8305,
 u'location': [-129.75341, 45.8305],
 u'longitude': -129.75341,
 u'orbitRadius': None}

Similar to an synchronous json data request, the asset information is returned as a list of json object. In the following example we will pull out the

In [15]:
ref_des_list = []
start_time_list = []
end_time_list = []
deployment_list = []
depth_list = []
lat_list = [] 
lon_list = [] 

for i in range(len(asset_info)):
    refdes = asset_info[i]['referenceDesignator']
    ref_des_list.append(refdes)

    deployment = asset_info[i]['deploymentNumber']
    deployment_list.append(deployment)

    start = asset_info[i]['eventStartTime']
    end = asset_info[i]['eventStopTime']

    try:
        start_time = datetime.datetime.utcfromtimestamp(start/1000.0)
        start_time_list.append(start_time)

        end_time = datetime.datetime.utcfromtimestamp(end/1000.0)
        end_time_list.append(end_time)

    except:
        end_time = datetime.datetime.utcnow()
        end_time_list.append(end_time)
        
    depth = asset_info[i]['location']['depth']
    depth_list.append(depth)
    lat = asset_info[i]['location']['latitude']
    lat_list.append(lat)
    lon = asset_info[i]['location']['longitude']
    lon_list.append(lon)
    

data_dict = {
    'refdes':ref_des_list,
    'deployment':deployment_list,
    'start_time':start_time_list,
    'end_time':end_time_list,
    'depth': depth_list,
    'latitutde':lat_list,
    'longitude':lon_list}

deployment_data = pd.DataFrame(data_dict, columns = ['refdes', 'deployment','start_time', 'end_time'])

deployment_data

refdes  deployment          start_time  \
0  RS03AXPS-PC03A-4A-CTDPFA303           1 2014-09-27 13:29:00   
1  RS03AXPS-PC03A-4A-CTDPFA303           2 2015-07-09 00:07:00   
2  RS03AXPS-PC03A-4A-CTDPFA303           3 2016-07-14 21:18:00   
3  RS03AXPS-PC03A-4A-CTDPFA303           4 2017-07-31 04:00:00   

                    end_time  
0 2015-07-09 00:00:00.000000  
1 2016-07-14 00:00:00.000000  
2 2017-07-30 14:00:00.000000  
3 2018-02-21 05:46:41.942697